Один из способов повысить эффективность взаимодействия банка с клиентами — отправлять предложение о новой услуге не всем клиентам, а только некоторым, которые выбираются по принципу наибольшей склонности к отклику на это предложение.

Задача заключается в том, чтобы предложить алгоритм, который будет выдавать склонность клиента к положительному или отрицательному отклику на предложение банка. Предполагается, что, получив такие оценки для некоторого множества клиентов, банк обратится с предложением только к тем, от кого ожидается положительный отклик.


In [141]:
import numpy as np

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import pickle

In [142]:
df = pd.read_csv('data/data.csv')

In [143]:
df.head()

,AGREEMENT_RK,TARGET,AGE,SOCSTATUS_WORK_FL,SOCSTATUS_PENS_FL,GENDER,CHILD_TOTAL,DEPENDANTS,PERSONAL_INCOME,LOAN_NUM_TOTAL,LOAN_NUM_CLOSED
0,59910150.0,0.0,49,1,0,1,2,1,5000.0,1.0,1.0
1,59910150.0,0.0,49,1,0,1,2,1,5000.0,1.0,1.0
2,59910230.0,0.0,32,1,0,1,3,3,12000.0,1.0,1.0
3,59910525.0,0.0,52,1,0,1,4,0,9000.0,2.0,1.0
4,59910803.0,0.0,39,1,0,1,1,1,25000.0,1.0,1.0


### Удаляем значения, где неизвестен таргет

In [144]:
df = df.dropna(subset='TARGET')

In [145]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15523 entries, 0 to 15522
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   AGREEMENT_RK       15523 non-null  float64
 1   TARGET             15523 non-null  float64
 2   AGE                15523 non-null  int64  
 3   SOCSTATUS_WORK_FL  15523 non-null  int64  
 4   SOCSTATUS_PENS_FL  15523 non-null  int64  
 5   GENDER             15523 non-null  int64  
 6   CHILD_TOTAL        15523 non-null  int64  
 7   DEPENDANTS         15523 non-null  int64  
 8   PERSONAL_INCOME    15523 non-null  float64
 9   LOAN_NUM_TOTAL     15523 non-null  float64
 10  LOAN_NUM_CLOSED    15523 non-null  float64
dtypes: float64(5), int64(6)
memory usage: 1.4 MB


### Разбиваем данные на тренировочную и тестовую часть в пропорции 80% к 20%, фиксируем `random_state = 42`.

In [146]:
X = df.drop(['TARGET', 'AGREEMENT_RK'], axis=1)
y = df['TARGET']

In [147]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Создаем пайплайн и обучаем модель

In [148]:
pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('model', LogisticRegression())
])

In [149]:
pipe.fit(X_train, y_train)

Pipeline(steps=[('scaler', StandardScaler()), ('model', LogisticRegression())])

In [150]:
y_pred = pipe.predict(X_test)

### Вычисляем метрики

In [151]:
print(f'Accuracy: {accuracy_score(y_test, y_pred)}')
print(f'Precision: {precision_score(y_test, y_pred)}')
print(f'Recall: {recall_score(y_test, y_pred)}')
print(f'F1: {f1_score(y_test, y_pred)}')

Accuracy: 0.8856682769726248
Precision: 0.3333333333333333
Recall: 0.002824858757062147
F1: 0.0056022408963585435


Целевая метрика для задачи - полнота, так как нам нужно найти максимум клиентов, кто может откликнуться на рекламу.
Но при этом точность не должна просесть, поэтому за ней тоже следим.

### Разбиваем тренировочные данные на `train` и `val` части в пропорции 3 к 1. Подберем порог

In [152]:
X_train_val, X_val, y_train_val, y_val = train_test_split(X_train, y_train, test_size=0.33, random_state=42)

In [153]:
d = {}

for i in np.arange(0, 1.01, 0.01):
    pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('model', LogisticRegression())
        ])
    pipe.fit(X_train_val, y_train_val)
    proba = pipe.predict_proba(X_val)[:, 1]

    classes = proba > i
    
    precision = precision_score(y_val, classes, zero_division=0)
    recall = recall_score(y_val, classes)

    d[i] = (precision, recall)

In [154]:
filtered_data = {i: k for i, k in d.items() if k[1] > 0.66}

In [155]:
best_metrics = max(filtered_data, key=lambda k: filtered_data[k][0])

In [156]:
best_metrics

0.12

### Итог, лучший порог - 0.12, вычислим остальные метрики

In [157]:
pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('model', LogisticRegression())
        ])

In [158]:
pipe.fit(X_train, y_train)
proba = pipe.predict_proba(X_test)[:, 1]

classes = proba > 0.12

In [159]:
print(f'Accuracy: {accuracy_score(y_test, classes)}')
print(f'Precision: {precision_score(y_test, classes ,zero_division=0)}')
print(f'Recall: {recall_score(y_test, classes)}')
print(f'F1: {f1_score(y_test, classes)}')

Accuracy: 0.5539452495974235
Precision: 0.15838303512259774
Recall: 0.6751412429378532
F1: 0.2565754159957058


### Как мы видим, метрики после валидации значительно изменились

### Интерпретация модели

In [160]:
pd.DataFrame({'Features': X_train.columns, 'Weights': pipe.named_steps['model'].coef_[0]}).sort_values(by='Weights')

,Features,Weights
8,LOAN_NUM_CLOSED,-0.446947
0,AGE,-0.302311
2,SOCSTATUS_PENS_FL,-0.047920
5,DEPENDANTS,-0.015480
3,GENDER,0.032437
4,CHILD_TOTAL,0.144431
1,SOCSTATUS_WORK_FL,0.208525
6,PERSONAL_INCOME,0.218516
7,LOAN_NUM_TOTAL,0.278578


### Сохранение модели в файл

In [161]:
with open('model.pickle', 'wb') as f:
    pickle.dump(pipe, f)

In [165]:
X.describe()

,AGE,SOCSTATUS_WORK_FL,SOCSTATUS_PENS_FL,GENDER,CHILD_TOTAL,DEPENDANTS,PERSONAL_INCOME,LOAN_NUM_TOTAL,LOAN_NUM_CLOSED
count,15523.000000,15523.000000,15523.000000,15523.000000,15523.000000,15523.000000,15523.000000,15523.000000,15523.000000
mean,40.400438,0.909360,0.134639,0.653997,1.098886,0.644850,13848.041638,1.387296,0.751594
std,11.607242,0.287105,0.341348,0.475709,0.996748,0.812663,8998.618992,0.794241,0.989253
min,21.000000,0.000000,0.000000,0.000000,0.000000,0.000000,24.000000,1.000000,0.000000
25%,30.000000,1.000000,0.000000,0.000000,0.000000,0.000000,8000.000000,1.000000,0.000000
50%,39.000000,1.000000,0.000000,1.000000,1.000000,0.000000,12000.000000,1.000000,0.000000
75%,50.000000,1.000000,0.000000,1.000000,2.000000,1.000000,17000.000000,2.000000,1.000000
max,67.000000,1.000000,1.000000,1.000000,10.000000,7.000000,250000.000000,11.000000,11.000000
